In [44]:
from collections import Counter
from enum import IntEnum
from collections import namedtuple
import numpy as np
import operator
import pandas as pd
import random

In [115]:
class card:

    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
    
    def __eq__ (self, other):
        return (self.value == other.value)

    def __ne__ (self, other):
        return (self.value != other.value)

    def __lt__ (self, other):
        return (self.value < other.value)

    def __le__ (self, other):
        return (self.value <= other.value)

    def __gt__ (self, other):
        return (self.value > other.value)

    def __ge__ (self, other):
        return (self.value >= other.value)
    
    def __add__ (self, int):
        return card(self.suit, self.value+1)
    
    def __sub__ (self, int):
        return card(self.suit, self.value-1)
    
    def __repr__(self):
        return f'{self.values[self.value]} of {self.suit}'

class card_array:

    def __init__(self, cards):
        self.cards = np.array(cards)
        self.suits = np.array([card.suit for card in cards])
        self.values = np.array([card.value for card in cards])

In [42]:
deck = []
suits = {"Diamonds": 0, "Clubs": 1, "Hearts":2, "Spades":3}
values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
for value in values.keys():
    for suit in suits.keys():
        deck.append(card(suit, value))

In [181]:
# hand = card_array(random.sample(deck, 7))
hand = card_array([card('Clubs', 14), card('Hearts', 14), card('Spades', 13), card("Clubs", 13), card("Hearts", 7), card("Diamond", 2), card("Diamond", 11), card("Clubs", 12), card("Clubs", 9)])

In [27]:
sorted_deck = sorted(deck, key=operator.attrgetter("suit"))

In [68]:
hand.suits

['Clubs', 'Hearts', 'Spades', 'Clubs', 'Hearts', 'Clubs', 'Clubs', 'Clubs']

In [87]:
values_count = Counter(hand.values)
values_count

Counter({14: 3, 13: 2, 10: 1, 11: 1, 12: 1, 9: 1})

In [213]:
def isDouble(cards_array):
    doubles = []
    values_counter = Counter(cards_array.values)
    for key in values_counter.keys():
        if values_counter[key] == 2:
            doubles.append(key)
    if len(doubles) != 0:
        return np.sort(doubles)
    return 0

def isTriple(cards_array):
    triples = []
    values_counter = Counter(cards_array.values)
    for key in values_counter.keys():
        if values_counter[key] == 3:
            triples.append(key)
    if len(triples) != 0:
        return np.sort(triples)
    return 0

def isFour(cards_array):
    values_counter = Counter(cards_array.values)
    for key in values_counter.keys():
        if values_counter[key] == 4:
            return key
    return 0

def isFullHouse(cards_array):
    triples = isTriple(cards_array)
    doubles = isDouble(cards_array)
    if np.all(triples != 0) and np.all(doubles != 0):
        return triples[-1]
    return 0

def isFlush(cards_array):
    suits_counter = Counter(cards_array.suits)
    for key in suits_counter.keys():
        if suits_counter[key] >= 5:
            return key
    return 0

def isStraight(cards_array):
    sorted_cards = np.array(sorted(set(cards_array.values)))
    sorted_cards -= np.arange(len(sorted_cards))
    same_counter = Counter(sorted_cards)
    for key in same_counter.keys():
        if same_counter[key] >= 5:
            return np.arange(same_counter[key]) + key
    return 0

def isStraightFlush(cards_array):
    flush = isFlush(cards_array)
    if np.all(flush != 0):
        return isStraight(card_array(cards_array.cards[cards_array.suits == flush]))
    return 0

def isRoyalFlush(cards_array):
    straight_flush = isStraightFlush(cards_array)
    if np.all(straight_flush != 0):
        if np.all(straight_flush[-5:] == np.arange(5)+10):
            return np.arange(5)+10
    return 0

def getBestHand(cards_array):
    combos = [isRoyalFlush(cards_array), isStraightFlush(cards_array), isFour(cards_array), isFullHouse(cards_array), isFlush(cards_array), isStraight(cards_array), isTriple(cards_array), isDouble(cards_array), cards_array.cards]
    for ranking, combo in enumerate(combos):
        if np.all(combo != 0):
            # if triple
            if ranking == 6:
                high_singles = np.sort(cards_array.values[cards_array.values != combos[0]])[-2:]
                return ranking, (combo[0], high_singles)
            # if double
            if ranking == 7:
                # if two double
                if len(combo) >= 2:
                    not_double_value1 = cards_array.values != combo[-1]
                    not_double_value2 = cards_array.values != combo[-2]
                    high_single = np.sort(cards_array.values[np.logical_and(not_double_value1, not_double_value2)])[-1]
                    return ranking, (combo[-2:], high_single)

                high_singles = np.sort(cards_array.values[cards_array.values != combo[-1]])[-3:]
                return ranking, (combo[-1], high_singles)
            # if singles only
            if ranking == 8:
                return ranking, np.sort(cards_array.values)[-5:]

            return ranking, combo 

In [212]:
getBestHand(hand)

14
[False False False False  True  True  True  True  True]


(7, (array([13, 14]), 12))

In [168]:
isFullHouse(hand)

0

In [100]:
np.array(hand.cards)[np.array(hand.suits) == "Clubs"]

array([Ace of Clubs, King of Clubs, 10 of Clubs, Jack of Clubs,
       Queen of Clubs, 9 of Clubs], dtype=object)

In [144]:
hand.cards is not 0

<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/nx/23kzl3_d08d89039y9hptpf40000gn/T/ipykernel_31199/2115302451.py:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  hand.cards is not 0


True

In [125]:
def do_nothing():
    y = 1+1

In [127]:
return1 = do_nothing()

In [135]:
isinstance(return1, NoneType)

TypeError: isinstance() arg 2 must be a type or tuple of types

In [210]:
np.array([True, False]) == np.array([True, False])

array([ True,  True])

In [148]:
np.all(p==0)

True

In [147]:
p = 0

In [5]:
import numpy as np
import random

In [12]:
num_players = 5
rand_pos = random.randint(0, num_players-1)

In [23]:
np.eye(1, num_players, rand_pos)

array([[0., 1., 0., 0., 0.]])

In [22]:
np.eye(1, num_players, rand_pos+1 if rand_pos+1 <= num_players else 0)

array([[0., 0., 1., 0., 0.]])

In [21]:
np.eye(1, num_players, rand_pos+2 if rand_pos+2 <= num_players else rand_pos+2-num_players)

array([[0., 0., 0., 1., 0.]])

In [20]:
rand_pos+2 if rand_pos+2 <= num_players else rand_pos+2-num_players

3

In [19]:
rand_pos+2 >= num_players

False